In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Оценка решения
Задачу выполнял в целях саморазвития, в формальном выставлении оценки не нуждаюсь

# Чтение данных
Загрузим данные и посмотрим несколько примеров записей

In [2]:
data = pd.read_csv("/kaggle/input/titanic/train.csv")
data.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


# Предобработка

Посмотрим типы данных. Поскольку предполагается использовать TPOT, который умеет работать только с числами, необходимо привести все данные к такому типу

In [3]:
data.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

Из Name ничего особо не вытащить - возраст и так известен, социальное положение приближенно описывается классом каюты и ценой билета. Номер билета также не даёт дополнительной информации. Посмотрим, сколько категорий в других - вдруг их имеет смысл перевести в числовые категории

In [4]:
for category in ['Sex', 'Ticket', 'Cabin', 'Embarked']:
    print(category, data[category].unique().size)

Sex 2
Ticket 681
Cabin 148
Embarked 4


Видно, что имеет смысл категоризовать пол и порт отправки. Посмотрим, какие вообще варианты есть:

In [5]:
print(data.Sex.value_counts())
print(data.Embarked.value_counts())

male      577
female    314
Name: Sex, dtype: int64
S    644
C    168
Q     77
Name: Embarked, dtype: int64


Теперь заменим эти варианты на числовые обозначения

In [6]:
data.Sex.replace({'male':0, 'female':1}, inplace=True)
data.Embarked.replace({'S':0, 'C':1, 'Q':2}, inplace=True)

Посмотрим, что теперь получилось. И заодно оценим, все ли данные заполнены, или где-то есть пропущенные значения

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    int64  
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    float64
dtypes: float64(3), int64(6), object(3)
memory usage: 83.7+ KB


Видно, что есть много пропущенных значений у Cabin (которая нам не интересна), пара у Embarked, и некоторая доля у Age. Чтобы не путаться, сначала удалим неинтересные типы колонок:

In [8]:
data = data.drop(['Name', 'Ticket','Cabin'], axis=1)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Sex          891 non-null    int64  
 4   Age          714 non-null    float64
 5   SibSp        891 non-null    int64  
 6   Parch        891 non-null    int64  
 7   Fare         891 non-null    float64
 8   Embarked     889 non-null    float64
dtypes: float64(3), int64(6)
memory usage: 62.8 KB


И теперь заполним пропущенные значения. Для Age используем медиану выборки, для Embarked запишем -1 (как ещё одно возможное значение)

In [9]:
data.Age.fillna(data.Age.median(), inplace=True)
data.Embarked.fillna(-1, inplace=True)

Посмотрим ещё раз данные:

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Sex          891 non-null    int64  
 4   Age          891 non-null    float64
 5   SibSp        891 non-null    int64  
 6   Parch        891 non-null    int64  
 7   Fare         891 non-null    float64
 8   Embarked     891 non-null    float64
dtypes: float64(3), int64(6)
memory usage: 62.8 KB


In [11]:
data.head(3)

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,0,22.0,1,0,7.2500,0.0
1,2,1,1,1,38.0,1,0,71.2833,1.0
2,3,1,3,1,26.0,0,0,7.9250,0.0


Видно, что для Age точность избыточная, как и для Embarked. Сделаем их целыми числами:

In [12]:
data = data.astype({'Age': 'int64', 'Embarked': 'int64'})

In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Sex          891 non-null    int64  
 4   Age          891 non-null    int64  
 5   SibSp        891 non-null    int64  
 6   Parch        891 non-null    int64  
 7   Fare         891 non-null    float64
 8   Embarked     891 non-null    int64  
dtypes: float64(1), int64(8)
memory usage: 62.8 KB


In [14]:
data.head(3)

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,0,22,1,0,7.2500,0
1,2,1,1,1,38,1,0,71.2833,1
2,3,1,3,1,26,0,0,7.9250,0


Теперь данные устраивают, можно переходить к созданию модели

# Подбор модели и её обучение

Сначала выделим в отдельные переменные все исходные данные и целевую переменную:

In [15]:
X = data.drop(['Survived', 'PassengerId'], axis=1)
Y = data['Survived']

In [16]:
X.head(3)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,0,22,1,0,7.2500,0
1,1,1,38,1,0,71.2833,1
2,3,1,26,0,0,7.9250,0


In [17]:
Y.head(3)

0    0
1    1
2    1
Name: Survived, dtype: int64

Так как данных сравнительно много, и непонятно, от чего зависит целевая переменная (Survived), то можно попробовать использовать модуль для автоматического подбора моделей - TPOT. TPOT может быть регрессором или классификатором. Так как целевой признак бинарный (выжил/нет), используем классификатор.

In [18]:
from tpot import TPOTClassifier

Для подбора моделей нужно задать параметры.
generations обозначает количество итераций - чтобы успеть досчитать модель до заката без суперкомпьютера, вместо рекомендованных 100 установим 10.
population_size - выборка на каждом шаге. Рекомендуется 100, возьмём 50.
n_jobs - колчиество задействованных ядер. Что скрывается за интерфейсом Kaggle - непонятно, поэтому попробуем использовать максимальное количество (-1)
verbosity - насколько подробно будет описываться процесс. Пусть показыват прогресс-бар, выберем "2"


In [19]:
tpot = TPOTClassifier(generations=10, population_size=50, n_jobs=-1, verbosity=2)

По всем известным данным выберем лучшую модель:

In [20]:
tpot.fit(X, Y)

Optimization Progress:   0%|          | 0/550 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.8305504990270542

Generation 2 - Current best internal CV score: 0.8305504990270542

Generation 3 - Current best internal CV score: 0.8305693302366455

Generation 4 - Current best internal CV score: 0.8350511581193899

Generation 5 - Current best internal CV score: 0.8361747536250078

Generation 6 - Current best internal CV score: 0.8372920720607621

Generation 7 - Current best internal CV score: 0.8406565815077522

Generation 8 - Current best internal CV score: 0.8406565815077522

Generation 9 - Current best internal CV score: 0.8406565815077522

Generation 10 - Current best internal CV score: 0.8406565815077522

Best pipeline: RandomForestClassifier(input_matrix, bootstrap=False, criterion=gini, max_features=0.8, min_samples_leaf=6, min_samples_split=8, n_estimators=100)


TPOTClassifier(generations=10, n_jobs=-1, population_size=50, verbosity=2)

Запомним лучшую модель (на случай, если output'ы будут стёрты):

RandomForestClassifier(XGBClassifier(input_matrix, learning_rate=0.5, max_depth=9, min_child_weight=9, n_estimators=100, n_jobs=1, subsample=0.7500000000000001, verbosity=0), bootstrap=False, criterion=entropy, max_features=0.05, min_samples_leaf=12, min_samples_split=17, n_estimators=100)

# Использование готовой модели

Читаем тестовые данные

In [21]:
data_test = pd.read_csv("/kaggle/input/titanic/test.csv")

In [22]:
data_test.head(3)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q


In [23]:
data_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


Делаем предобработку по аналогии с обучающим набором данных

In [24]:
data_test.Sex.replace({'male':0, 'female':1}, inplace=True)
data_test.Embarked.replace({'S':0, 'C':1, 'Q':2}, inplace=True)
data_test.Age.fillna(data_test.Age.median(), inplace=True)
data_test.Embarked.fillna(-1, inplace=True)
data_test = data_test.drop(['PassengerId', 'Name', 'Ticket','Cabin'], axis=1)
data_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    418 non-null    int64  
 1   Sex       418 non-null    int64  
 2   Age       418 non-null    float64
 3   SibSp     418 non-null    int64  
 4   Parch     418 non-null    int64  
 5   Fare      417 non-null    float64
 6   Embarked  418 non-null    int64  
dtypes: float64(2), int64(5)
memory usage: 23.0 KB


Видно, что цена билета одного пассажира не указана. Заполним её медианой (по аналогии с возрастом)

In [25]:
data_test.Fare.fillna(data_test.Fare.median(), inplace=True)

Поправим типы данных:

In [26]:
data_test = data_test.astype({'Age': 'int64', 'Embarked': 'int64'})

In [27]:
data_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    418 non-null    int64  
 1   Sex       418 non-null    int64  
 2   Age       418 non-null    int64  
 3   SibSp     418 non-null    int64  
 4   Parch     418 non-null    int64  
 5   Fare      418 non-null    float64
 6   Embarked  418 non-null    int64  
dtypes: float64(1), int64(6)
memory usage: 23.0 KB


Сделаем само предсказание

In [28]:
result = tpot.predict(data_test)

Посмотрим, что напредсказывалось

In [29]:
print('Размерность матрицы ответов', result.shape)
print('Все варианты ответов - проверяем, что только 0 и 1', np.unique(result))
print('Смотрим, сколько "0" и сколько "1"', np.bincount(result))

Размерность матрицы ответов (418,)
Все варианты ответов - проверяем, что только 0 и 1 [0 1]
Смотрим, сколько "0" и сколько "1" [276 142]


Запись ответов

In [30]:
data_to_make_ans = pd.read_csv("/kaggle/input/titanic/test.csv")

In [31]:
df_result = pd.DataFrame({'PassengerId': data_to_make_ans.PassengerId, 'Survived': result})

In [32]:
df_result.head(3)

,PassengerId,Survived
0,892,0
1,893,0
2,894,0


In [33]:
df_result.to_csv("submission_tpot_upgraded.csv", index=False)

Итоговый результат на тестовой выборке (в рейтинге Kaggle) - 0.76555.